In [1]:
!pip install git+https://github.com/LIAAD/yake
!pip install nltk

  Cloning https://github.com/LIAAD/yake to c:\users\mrwnh\appdata\local\temp\pip-req-build-83o9_f4_
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake 'C:\Users\mrwnh\AppData\Local\Temp\pip-req-build-83o9_f4_'

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import yake
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrwnh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrwnh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def preprocess_text(text):
    # Tokenisasi teks menjadi kata-kata
    words = word_tokenize(text.lower())

    # Penghapusan tanda baca dan karakter non-abjad
    words_filtered = [word for word in words if word.isalnum()]

    # Penghapusan kata-kata stopword
    stop_words = set(stopwords.words("indonesian"))
    words_filtered = [word for word in words_filtered if word not in stop_words]

    # Stemming kata-kata untuk mendapatkan bentuk kata dasar
    ps = PorterStemmer()
    words_stemmed = [ps.stem(word) for word in words_filtered]

    return " ".join(words_stemmed)

def yake_keywords(text, max_keywords=5):
    # Inisialisasi ekstraktor YAKE
    custom_kw_extractor = yake.KeywordExtractor(n=1, top=max_keywords, features=None)

    # Ekstraksi kata kunci menggunakan YAKE
    keywords = custom_kw_extractor.extract_keywords(text)

    return [keyword for keyword, score in keywords]

def read_abstract_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [4]:
# Folder path for the abstracts
folder_path = "abstrak/"

# Create an empty DataFrame to store the keyword extraction results
result_df = pd.DataFrame(columns=["File", "Abstrak", "Preprocessed Text", "Keywords"])

# Extract keywords for each abstract in each file
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)

        # Read the contents of the abstract file
        text = read_abstract_from_file(file_path)

        # Divide the text into parts based on "Kata Kunci:"
        abstracts = text.split("Kata Kunci:")

        # Extract keywords for each abstract in the file
        for idx, abstract in enumerate(abstracts[1:], start=1):
            # Separate abstract and keywords section
            abstract_parts = abstract.split("\n")

            # Preprocess the text in the abstract
            preprocessed_text = preprocess_text(abstract_parts[0])

            # Extract keywords using YAKE
            keywords = yake_keywords(preprocessed_text)

            # Add the extraction results to the DataFrame
            result_df = pd.concat([result_df, pd.DataFrame({
                "File": [filename],
                "Abstrak": [idx],
                "Preprocessed Text": [preprocessed_text],
                "Keywords": [", ".join(keywords)]
            })])

# Save the DataFrame to a CSV file
result_csv_path = "keyword_abstrak.csv"
result_df.to_csv(result_csv_path, index=False)

In [5]:
# Read the saved CSV file
read_result_df = pd.read_csv(result_csv_path)

# Print the DataFrame read from the CSV
print("Hasil Ekstraksi dari File CSV:")
print(read_result_df)

Hasil Ekstraksi dari File CSV:
           File  Abstrak                                  Preprocessed Text  \
0  abstrak2.txt        1      prediksi studi case base reason cosin similar   
1  abstrak3.txt        1  sistem pendukung keputusan prestasi analyt hie...   
2  abstrak4.txt        1  sistem pendukung keputusan weight product hype...   
3  abstrak5.txt        1                 sistem informasi penjurusan angket   

                                          Keywords  
0             prediksi, similar, studi, case, base  
1  sistem, process, pendukung, keputusan, prestasi  
2    sistem, jurusan, pendukung, keputusan, weight  
3            sistem, angket, informasi, penjurusan  
